In [11]:
import pandas as pd
import numpy as np
import spacy
from pp import pp

In [12]:
CHECK_WORDS = ['not', 'non', 'other', 'other than', 'besides',]
NUMBERS = ['2', '2A', '2B', '2a', '2b']

In [13]:
def dis(a, b):
    dot = np.dot(a, b)
    norma = np.linalg.norm(a)
    normb = np.linalg.norm(b)
    cos = dot / (norma * normb)
    return cos

In [23]:
#create centroid
def cent(t, idfs):
    n = np.zeros((len(t), len(t.vector)))
    for i in range(len(t)):
        if str(t[i]) in CHECK_WORDS:
            n[i] = n[i] + ((idfs[str(t[i])])*(5*t[i].vector))
        elif str(t[i]) in NUMBERS:
            n[i] = n[i] + ((idfs[str(t[i])])*(-5*t[i].vector))
            #n[i] = n[i] + (-5*t[i].vector)
        else:
            n[i] = n[i] + ((idfs[str(t[i])])*(2*t[i].vector))
            #n[i] = n[i] + (2*t[i].vector)
                        
    return n.mean(axis = 0)

In [15]:
#finding idf
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10((N + .01)/ float(val))
        
    return idfDict    

In [16]:
#create the model
model = spacy.load("en_core_web_lg")

In [31]:
df = pd.read_csv("test.csv",header = None)
df = df.fillna(value = "")

In [32]:
results = []
result = {}


In [33]:
for sent1 in df[0]:
    for sent2 in df[1]:
        #result['Field1'] = sent1.strip()
        #result['Field2'] = sent2.strip()
        #print(sent2)
        #result["Similarity"] = compare(sent1, sent2)
        
        if (len(sent2)>3):
            pre_res = pp([sent1.strip(), sent2.strip()])


            test1 = " ".join(pre_res[0])
            test2 = " ".join(pre_res[1])
            
            #creating bag of words for idf
            
            bow1 = pre_res[0]
            bow2 = pre_res[1]
            
            wordset = {}
            wordset = set(bow1).union(set(bow2))
            
            wordDict1 = dict.fromkeys(wordset, 0) 
            wordDict2 = dict.fromkeys(wordset, 0) 
                        
            for word in bow1:
                wordDict1[word]+=1
    
            for word in bow2:
                wordDict2[word]+=1
                
            idfs = computeIDF([wordDict1, wordDict2])
            
            
            t1 = model(test1)
            t2 = model(test2)
            c1 = cent(t1, idfs)
            c2 = cent(t2, idfs)
            result = {}
            result['Field1'] = sent1.strip()
            result["Processed Field1"] = test1
            result["Processed Field2"] = test2
            result['Field2'] = sent2.strip()
            result["Similarity"] = dis(c1, c2)
            results.append(result)

            

/home/venkatesh/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: Mean of empty slice.
  


In [34]:
len(results)

81

In [35]:
results_df = pd.DataFrame(results)

In [36]:
results_df.iloc[10,0]


'Cash inflows: Transactions backed by Level 2A assets'

In [37]:
results_df.to_excel("result21.xlsx")